In [1]:
library("dplyr")
library("corrplot")
library("corpcor")
library("GPArotation")
library("psych")
library("IDPmisc") 

Warning message:
"package 'dplyr' was built under R version 4.1.2"


ERROR: Error: package or namespace load failed for 'dplyr' in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 namespace 'rlang' 0.4.11 is already loaded, but >= 1.0.0 is required


In [ ]:
house <- read.csv('C:/Users/Marcy/Documents/Entity Coursework/Final Project/house.csv')
x <- read.csv("C:/Users/Marcy/Documents/Entity Coursework/Final Project/x.csv")
y <- read.csv("C:/Users/Marcy/Documents/Entity Coursework/Final Project/y.csv")

In [ ]:
tail(house)

#### Create a model that uses SalePrice as the response variable and the other columns as potential predictor variables. Start by creating a function called FitAll that creates a linear model of all predictor variables, which have been recoded. 

In [ ]:
FitAll = lm(SalePrice ~ ., data = house)
summary(FitAll)

#### Features significantly tied to house prices 
#Some of the individual predictors are significant, and they're what you might expect: Overall quality, overall conditin, year built, roofing material, first and second floor square footage, basement full bath, functional, garage cars. Zoning, lot area, Screen porch and sale condition are important to a lesser degree. 
 

#### Hybrid Stepwise - Forward and Backward Selection
#Starting with no predictors, using the mean for HousePrice only.

In [ ]:
fitstart = lm(SalePrice~1, data = house)
step(fitstart, direction="both", scope=formula(FitAll))

#### The AIC with no predictors at all is 32946.74. Adding anything at all will bring the AIC down, but the second iteration shows that the best way to improve the model is to add OverallQual of the house as a predictor. 
Iterating through each of the independent variables, finally at the end, we achieve an AIC of 29376.69. The predictor variables that most improve the model are SalePriceRange + OverallQual + GrLivArea + YearBuilt + 
    OverallCond + GarageCars + FireplaceQu + RoofMatl + BsmtFullBath + 
    BldgType + KitchenQual + LotArea + Functional + GarageType + 
    HeatingQC + ScreenPorch + SaleCondition + MSZoning + GarageYrBlt + 
    KitchenAbvGr + Street + WoodDeckSF + Fireplaces + BsmtQual + 
    X1stFlrSF + BsmtFinType1 + GarageFinish + FullBath + LotFrontage + 
    BsmtHalfBath




#### This does bring to mind more questions: do the importance of these features to homebuyers in this area hold true for homebuyers in other parts of the country?  Is a pool important to homebuyers across the (diving) board, so to speak? 
Removing the particular neighborhoods listed in this dataset may change the outcome of the regression model. Is it even necessary to remove the neighborhoods for my purposes. But to generalize this across the nation, specific locations or neighborhoods would not be useful. Also, is a heating system as important to homebuyers in Arizona as it would be to those settling down in Michigan? This dataset is located in Iowa, so heating systems are important. In Florida, basements are rare. Would they still boost a home's sale price? 

#### Exploratory Factor Analysis
There are three assumptions: sample size, absence of multicollinearity and some relationship between the independent variables. 

#### I'm not going to subset the data, so I'll go straight to a cor matrix to test for the assumption of the absence of multicollinearity.

In [ ]:
subset = subset(house, select = -c(SalePrice))
houseCor <- cor(subset)
View(round(houseCor, 2))

#### look for any correlations that are higher than .9. This would indicate really high multicollinearity, and if there's an item that has a correlation of .9, I  will most likely want to remove that item

#### I don't see any higher than .55  
to doublecheck my findings from the correlation matrix, I'll run a Bartlett's test.

In [ ]:
cortest.bartlett(subset)

#### check determinants, which is another measure of how the variables relate to each other. 

#### Initial pass to determine approximate number of factors. Because I didn't subset, I'm going to try 20 factors. 

In [ ]:
pcZ <- principal(subset, nfactors = 20, rotate = "none")
pcZ
#pcZ.loadings.df <- data.frame(pcZ$loading)
#write.csv(pcZ.loadings.df, "ss_loadings.csv")
#write.csv(data.frame(pcZ$loading > 1.0), "loadings_above_one.csv")

#### R has tried to break down the data into 20 factors. On the first pass is the SS loadings on the bottom, which contains the eigenvalues. 
The larger the eigenvalue, the more likely the factor is important. Typically there is a cutoff of 1, so any factors with an eigenvalue > 1 should be examined. 
In this table, all the PC values are greater than 1.  

#### Examine the scree plot for a more visual return of eigenvalues generated by the model. 

In [ ]:
plot(pcZ$values, type="b")

#### There's a sheer dropoff after 10-12 factors, down to 50, where the trendline continues its descent. I'll go with top 20 factors and test that assumption, to see if the model fit improves with 12 factors since that's about where the rest flow together.

In [ ]:
pcZ2 <- principal (subset, nfactors = 20, rotate = "none")
pcZ2

#### examining residuals to determine model fit
 The basic idea behind this test is that the model fits your data very well if there is very little difference between the correlation matrix and the loadings generated through your model. The difference between them is known as the residual. A general rule of thumb is that you have good model fit if the percentage of large residuals (over .05) is less than 50%. In order to make all this easier, you will go through a series of steps. The first line creates your residuals, using the factor.residuals() function. The argument it takes are your correlation matrix and the loadings from your most recent factor analysis model.

In [ ]:
residuals <- factor.residuals(subset, pcZ2$loadings)
residuals <- as.matrix(residuals[upper.tri(residuals)])
largeResid <- abs(residuals) > .05
sum(largeResid)
sum(largeResid/nrow(residuals))

#### 89% of the residuals are large.
This is over 50%, so having 12 factors is a pretty good model fit for the data.

#### Oblique Rotation
Oblimin is the most commonly used type of oblique rotation and I'll keep 12 fators because an examination of the residuals showed that was a good fit for the data. 

In [ ]:
pcZ3 <- principal(subset, nfactors=20, rotate = "oblimin")
pcZ3

#### exporting to take a look at the pattern matrix, highlight the values that load highly (anything above .3-.4 loads on that factor)
#pcZ3 <- write.csv(pcZ3, "C:/Users/Marcy/Documents/Entity Coursework/Final Project/pcZ3.csv")

In [ ]:
print.psych(pcZ3, cut = .3, sort = TRUE)

### import unemployment numbers to illustrate Iowa's spike in unemployment during the stay at home order in 2020

In [ ]:
unemployment <- read.csv ("C:/Users/Marcy/Documents/Entity Coursework/Final Project/Unemployment Insurance Claims.csv")

In [ ]:
head(unemployment)

In [ ]:
library("ggplot2")
ggplot(subset(unemployment, Geography != 'Iowa'), aes(x=Dates, y='Unemployment Insurance Claims')) +
    geom_line(aes(color= 'gray', group = Geography)) +
    geom_line(data = subset(unemployment, Geography == 'Iowa'), size=3, color = 'blue') +
    theme_bw()